## Finding the best day to make a recurring investment
### (If it actually does make a difference)

First, importing all the libraries this will use:

In [14]:
import datetime
import calendar
import pandas as pd
import plotly.express as px
import yfinance as yf
import sys

First we need to create some helper functions to make things simpler. 

In [8]:
#convert dates to days of week
def dayOfWeek(date):
    return calendar.day_name[datetime.datetime.strptime(date, '%Y-%m-%d').weekday()]

#get data on the selected stock from yahoo finance
def getStockData(ticker):
    df = yf.Ticker(ticker).history(period="5y")
    df.reset_index(inplace=True)
    df.rename(columns = {'index' : 'Date'})
    return df

#calculates average price between the opening and closing price for more realistic returns
def midPrice(data, ind):
    return (data.iloc[ind]['Open']+data.iloc[ind]['Close'])/2

#calculate returns over the chosen number of days
def calcReturn(data, ind, days):
    r = (midPrice(data, ind+days) - midPrice(data, ind))/midPrice(data, ind)
    return r

#create the series of returns for the entire length of the data
def populateReturns(stock_df, days):
    returns = []
    for i in range(0, len(stock_df)-days):
        returns.append(calcReturn(stock_df, i, days))
    return returns + [None] * days



Finally, the above functions are combined to create a box and whisker plot using plotly. Plotly is especially nice for this application because it lets us hover to see exact numbers. The downside is that it needs a running notebook or has to be exported. If you want to export as an image or html file, just uncomment the appropriate line. 

In [18]:
def createChart(ticker):
    stock_df = getStockData(ticker)

    returns_df = pd.DataFrame()
    returns_df['Date'] = stock_df['Date']

    mid_price = []
    weekdays = []
    third = []
    for i in range(0, len(returns_df)):
        weekdays.append(dayOfWeek(str(stock_df.iloc[i]['Date'])[:10]))
        mid_price.append(midPrice(stock_df, i)) 
        if int(str(stock_df.iloc[i]['Date'])[8:10]) < 11:
            third.append('Early')
        elif (11 < int(str(stock_df.iloc[i]['Date'])[8:10]) < 21):
            third.append('Middle')
        else:
            third.append('Late')
    returns_df['Weekday'] = weekdays
    returns_df['Price'] = mid_price
    returns_df['Time_of_month'] = third
    #could add or change the below line to see different time returns
    returns_df['Returns_1y'] = populateReturns(stock_df, 252)

    fig = px.box(returns_df, x = 'Weekday', y='Returns_1y', color='Time_of_month', title='Returns by Day and Time of Month')
    fig.update_xaxes(categoryorder='array', categoryarray= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    fig.show()

    #fig.write_image(f"returns_{ticker}.png")
    #fig.write_html(f"returns_{ticker}.html", include_plotlyjs='cdn')

Now just pick a stock that you are interested in and find out if there is a better day to place an order (but remember that historical returns are not always a good predictor of future returns):

In [19]:
#stock to examine
ticker = "SPY"

createChart(ticker)